In [10]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 1. 네 꼭짓점의 중앙 좌표 계산
def calculate_corner_centroid(polygon):
    if polygon and polygon.geom_type == 'Polygon':
        coords = list(polygon.exterior.coords)
        if len(coords) >= 4:
            # 첫 네 꼭짓점의 좌표 추출
            first_four_coords = coords[:4]
            # x와 y 좌표의 평균 계산
            x_avg = sum(coord[0] for coord in first_four_coords) / 4
            y_avg = sum(coord[1] for coord in first_four_coords) / 4
            return Point(x_avg, y_avg)  # Shapely의 Point 객체로 반환
    return None

# 2. 주요 코드 실행
try:
    # 파일 경로 설정 (수정 필요)
    file_path = r'C:\Users\82102\OneDrive\첨부 파일\경희대학교\강의\3-1\물류관리\물류관리_텀프\(B100)국토통계_인구정보-총 인구 수(전체)-(격자) 100M_경기도 영통구_202404\nlsp_021001001.shp'

    # Shapefile 읽기
    population = gpd.read_file(file_path, encoding='UTF-8')
    print("Shapefile 데이터 로드 성공!")

    # 네 꼭짓점의 중앙 좌표 계산 (Shapely Point 객체로 저장)
    population['corner_centroid'] = population['geometry'].apply(calculate_corner_centroid)

    # 중심 좌표를 GeoSeries로 변환 (좌표계 변환을 위해 필요)
    centroid_series = gpd.GeoSeries(population['corner_centroid'], crs=population.crs)

    # 중심 좌표를 WGS84로 변환
    centroid_series_wgs84 = centroid_series.to_crs(epsg=4326)

    # 변환된 좌표를 새로운 칼럼에 추가
    population['corner_centroid_wgs84'] = centroid_series_wgs84.apply(lambda p: (p.y, p.x))

    # 새로운 DataFrame 생성: 고유번호, 인구 수(val), 중심 좌표, 위경도 좌표
    result_df = population[['gid', 'val', 'corner_centroid_wgs84']].copy()

    # 결과 출력
    print(result_df)

except Exception as e:
    print(f"오류 발생: {e}")



Shapefile 데이터 로드 성공!
           gid    val                     corner_centroid_wgs84
0     다사620212  388.0   (37.28941691921775, 127.07185367954239)
1     다사605154  493.0   (37.23707471317087, 127.05523890128033)
2     다사610161  215.0    (37.24340540804831, 127.0608391666334)
3     다사618165  402.0   (37.24704403594163, 127.06983843085936)
4     다사619165  204.0   (37.24704812659253, 127.07096595523109)
...        ...    ...                                       ...
2986  다사619205    NaN  (37.283103215286786, 127.07076136404814)
2987  다사596152    NaN   (37.23523341489345, 127.04510338123616)
2988  다사629185    NaN   (37.26511604078594, 127.08214164830109)
2989  다사590160    NaN  (37.242418257346856, 127.03829533848194)
2990  다사622176    NaN   (37.25697550991766, 127.07429274670774)

[2991 rows x 3 columns]


In [13]:
import pandas as pd
import numpy as np
from math import sqrt

# Assuming result_df and candidate_sites_df are DataFrames
print("Type of result_df:", type(result_df))
print("Type of candidate_sites_df:", type(candidate_sites_df))

# Function to calculate Euclidean distance
def calculate_euclidean_distance(coord1, coord2):
    return sqrt((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2)

# Distance threshold (d-value)
d = 0.5  # Example threshold distance (~1 km)

# Prepare gid and 시설명
gid_list = result_df['gid']
facility_list = candidate_sites_df['시설명'].tolist()

# Create an empty matrix
matrix = pd.DataFrame(0, index=gid_list, columns=facility_list)

# Populate the matrix
for i, gid_row in result_df.iterrows():
    gid = gid_row['gid']
    val = gid_row['val']
    centroid = gid_row['corner_centroid_wgs84']
    
    if centroid is None or pd.isna(val):
        continue  # Skip rows with no centroid or NaN values
    
    for _, site_row in candidate_sites_df.iterrows():
        facility = site_row['시설명']
        latitude = site_row['위도']
        longitude = site_row['경도']
        site_coord = (latitude, longitude)
        
        # Calculate Euclidean distance
        distance = calculate_euclidean_distance(centroid, site_coord)
        
        # Assign value to matrix if within distance threshold
        if distance <= d:
            matrix.at[gid, facility] = val * distance

# Display the resulting matrix
print(matrix)



Type of result_df: <class 'pandas.core.frame.DataFrame'>
Type of candidate_sites_df: <class 'pandas.core.frame.DataFrame'>


C:\Users\82102\AppData\Local\Temp\ipykernel_14260\1455359442.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.481439837132136' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matrix.at[gid, facility] = val * distance
C:\Users\82102\AppData\Local\Temp\ipykernel_14260\1455359442.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.013887099880346' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matrix.at[gid, facility] = val * distance
C:\Users\82102\AppData\Local\Temp\ipykernel_14260\1455359442.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.093607406620595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


          망포1동 행정복지센터  매탄3동 행정복지센터  매탄2동 행정복지센터  광교1동 행정복지센터  영통종합사회복지관  \
gid                                                                       
다사620212    20.481440    17.013887    12.093607     9.492768  15.212332   
다사605154     2.764841    11.213438    16.729035    26.469473  12.754845   
다사610161     1.205192     4.716330     6.423712    10.465464   3.861495   
다사618165     5.197246    11.375973    12.930408    19.550006   3.321038   
다사619165     2.803605     5.990427     6.723594    10.028370   1.480441   
...               ...          ...          ...          ...        ...   
다사619205     0.000000     0.000000     0.000000     0.000000   0.000000   
다사596152     0.000000     0.000000     0.000000     0.000000   0.000000   
다사629185     0.000000     0.000000     0.000000     0.000000   0.000000   
다사590160     0.000000     0.000000     0.000000     0.000000   0.000000   
다사622176     0.000000     0.000000     0.000000     0.000000   0.000000   

          영통2동 행정복지센터  영